<a href="https://colab.research.google.com/github/samas-it-services/open-course-delta-lake/blob/feature%2Fblockchain-via-delta-lake/Delta_Lake_with_Apache_Spark_Time_Travel_and_Versioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install required libraries
!pip install -q pyspark
#Supports delta in colab environment
!pip install -q delta-spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
def _create_delta_spark():
  from pyspark.sql import SparkSession
  from delta import configure_spark_with_delta_pip
  builder = SparkSession.builder.appName("DeltaLakeApp") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
  .config("spark.jars.packages","io.delta:delta-core_2.12:2.0.0")
  return configure_spark_with_delta_pip(builder).getOrCreate()

spark = _create_delta_spark()

In [3]:
def _enable_sparkui(port=4040):
    from google.colab import output
    return output.serve_kernel_port_as_window(port, path='/jobs/index.html')

_enable_sparkui()

<IPython.core.display.Javascript object>

# Read sample data
We read sample data from California Housing Transactions dataset

In [4]:
# read sample data
df = spark.read.options(inferSchema=True, header=True).csv("/content/sample_data/california_housing_train.csv")
df.printSchema()

# root
#  |-- longitude: double (nullable = true)
#  |-- latitude: double (nullable = true)
#  |-- housing_median_age: double (nullable = true)
#  |-- total_rooms: double (nullable = true)
#  |-- total_bedrooms: double (nullable = true)
#  |-- population: double (nullable = true)
#  |-- households: double (nullable = true)
#  |-- median_income: double (nullable = true)
#  |-- median_house_value: double (nullable = true)p

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



# Create a database

In [5]:
#create a database
spark.sql("create database delta_demo")
spark.sql("use delta_demo")

DataFrame[]

# Write csv data as delta table

In [6]:
# write csv data as delta table
df.write.mode("overwrite").format("delta").saveAsTable("df_delta")

# Display data

In [7]:
df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

# Make a schema Change
Create a new column in housing_df_delta dataframe with the name "median_house_value_new" and its values as "median_house_value"*1.1

In [8]:
#read delta table
df_delta = spark.read.table("delta_demo.df_delta")

# create a new column in housing_df_delta dataframe with the name "median_house_value_new" and its values as "median_house_value"*1.1
df_delta = df_delta.withColumn("median_house_value_new", df_delta["median_house_value"] * 1.1)
df_delta.show(1)


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|median_house_value_new|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+----------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|               73590.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+----------------------+
only showing top 1 row



In [9]:
df_delta.write\
.option("mergeSchema", "true")\
.mode("append").format("delta").saveAsTable("df_delta")

In [10]:
df_delta_after_append = spark.read.table("delta_demo.df_delta")
df_delta_after_append.show(20)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|median_house_value_new|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+----------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|                  NULL|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|                  NULL|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|                  NULL|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|         

# Show Delta table change history

In [11]:
#Load Delta table
from delta import DeltaTable
deltaTable = DeltaTable.forName(spark, "delta_demo.df_delta")

#View history
history = deltaTable.history().show(truncate =False)


+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                           |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------

In [12]:
#read version 0
version0DF = spark.read.format("delta")\
  .option("versionAsOf", 0)\
  .load("/content/spark-warehouse/delta_demo.db/df_delta/")

version0DF.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

# Read version 1

In [26]:
import ipywidgets as widgets
from IPython.display import display, Markdown
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws

# Initialize Spark session with Delta Lake
builder = SparkSession.builder \
    .appName("DeltaTableUI") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = builder.getOrCreate()

# Define the Delta table
delta_table = DeltaTable.forName(spark, "delta_demo.df_delta")

# Get all versions of the Delta table
history_df = delta_table.history()

# Convert timestamp to string and create display_text column
history_df = history_df.withColumn("display_text",
    concat_ws(": ", col("version").cast("string"),
                   concat_ws(" - ", col("timestamp").cast("string"), col("operation").cast("string")))
)

history_options = history_df.select("version", "display_text").collect()
dropdown_options = [(row['display_text'], row['version']) for row in history_options]

# Create form fields
version_dropdown = widgets.Dropdown(
    options=dropdown_options,
    description="Version:",
    value=None
)
submit_button = widgets.Button(description="Submit")

# Function to handle form submission
def display_version(b):
    if version_dropdown.value is None:
        print("Please select a version.")
        return

    selected_version = version_dropdown.value
    try:
        # Read the Delta table for the selected version
        # df = spark.read.format("delta").option("versionAsOf", selected_version).table("delta_demo.df_delta")
        df = spark.read.format("delta").option("versionAsOf", selected_version).load("/content/spark-warehouse/delta_demo.db/df_delta/")
        print(f"display version {selected_version}")
        df.show(truncate=False)
    except Exception as e:
        print(f"Error displaying version {selected_version}: {e}")

# Attach the display function to the button
submit_button.on_click(display_version)

# Display the UI form with a description
display(Markdown("""
# Delta Table Version Viewer

Use this form to view the contents of the Delta table at different versions.
Select a version from the dropdown and click "Submit" to display the contents of the table for the selected version.

**Note:** Viewing this data does not change the actual content of the Delta table.
"""))

print("Displaying all versions of the Delta table")
history_df.show(10)

# Display the form
display(version_dropdown, submit_button)



# Delta Table Version Viewer

Use this form to view the contents of the Delta table at different versions. 
Select a version from the dropdown and click "Submit" to display the contents of the table for the selected version.

**Note:** Viewing this data does not change the actual content of the Delta table.


Displaying all versions of the Delta table
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|        display_text|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+--------------------+
|      1|2024-06-24 01:35:...|  NULL|    NULL|               WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          0|  Serializable|        false|{numFiles -> 1, n...|        NULL|Apache-Spark/3.5....|1: 2024-06-24 01:...|
|      0|2024-06-24 01:35:...|  NULL|

Dropdown(description='Version:', options=(('1: 2024-06-24 01:35:56.4 - WRITE', 1), ('0: 2024-06-24 01:35:31.18…

Button(description='Submit', style=ButtonStyle())

display version 0
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31  |34.19   |15.0              |5612.0     |1283.0        |1015.0    |472.0     |1.4936       |66900.0           |
|-114.47  |34.4    |19.0              |7650.0     |1901.0        |1129.0    |463.0     |1.82         |80100.0           |
|-114.56  |33.69   |17.0              |720.0      |174.0         |333.0     |117.0     |1.6509       |85700.0           |
|-114.57  |33.64   |14.0              |1501.0     |337.0         |515.0     |226.0     |3.1917       |73400.0           |
|-114.57  |33.57   |20.0              |1454.0     |326.0         |624.0     |262.0     |1.925        |65500.0           |
|-114.